# VAE data preparation

We are going to prpeare the data to me able to use it on the vae model of https://github.com/evasnow1992/DeepGenerativeModelLINCS/tree/master

**Steps:**
1. There a multiple drug signatures for one combination of cell line - drug. I am going to use the mean
2. Make a list with all the drugs and their respective drug signatures chosen
3. Pass the drug signaures into the model 

## 1) Choose most optimal drug signature

#### 1.1) Drugcomb 

In [7]:
import pandas as pd
import gzip, shutil
import os
import pkg_resources

from cmapPy.pandasGEXpress.parse import parse

In [12]:
import pandas as pd
import gzip

# gene info 
file_path = r"C:\Users\K6502\OneDrive\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_gene_info_2017-03-06.txt.gz"
with gzip.open(file_path, 'rt') as f:
    gene_info = pd.read_csv(f, sep='\t')

# cell info 
file_path = r"C:\Users\K6502\OneDrive\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_cell_info_2017-04-28.txt.gz"
with gzip.open(file_path, 'rt') as f:
    cell_info = pd.read_csv(f, sep='\t')

# pert info 
file_path = r"C:\Users\K6502\OneDrive\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_pert_info.txt.gz"
with gzip.open(file_path, 'rt') as f:
    pert_info = pd.read_csv(f, sep='\t')

# inst info 
file_path = r"C:\Users\K6502\OneDrive\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_inst_info_2017-03-06.txt.gz"
with gzip.open(file_path, 'rt') as f:
    inst_info = pd.read_csv(f, sep='\t')

# sig info 
file_path = r"C:\Users\K6502\OneDrive\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_sig_info_2017-03-06.txt.gz"
with gzip.open(file_path, 'rt') as f:
    sig_info = pd.read_csv(f, sep='\t')

# sig metrics 
file_path = r"C:\Users\K6502\OneDrive\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_sig_metrics_2017-03-06.txt.gz"
with gzip.open(file_path, 'rt') as f:
    sig_metrics = pd.read_csv(f, sep='\t')


In [16]:
#summary_drugcomb=pd.read_csv(r"C:\Users\K6502\OneDrive\Desktop\TFM\datos\CLUE_data\CMAPS\combinations_cmaps_drugcomb_duplicates.csv")
summary_drugcomb=pd.read_csv(r"C:\Users\K6502\OneDrive\Desktop\TFM\datos\CLUE_data\CMAPS\combinations_cmaps_drugcomb_final.csv")

In [17]:
print(summary_drugcomb.shape) ##bien 

(1863, 42)


In [19]:
summary_drugcomb.head()

,block_id,drug_drug1_id,drug_drug2_id,cell_line_id,conc_r_unit,conc_c_unit,synergy_zip,synergy_loewe,synergy_hsa,synergy_bliss,...,smiles_drug2,molecular_formula_drug2,drugbank_id_drug2,kegg_id_drug2,cmaps_name_drug2,cell_line_name,cell_disease_id,tumor_location_cosmic,tumor_location_cellosaurus,disease_cui
0,4641,1,2,4,uM,uM,-1.832288,-26.978617,-8.166073,-4.944960,...,CC1(CCCN1)C2=NC3=C(C=CC=C3N2)C(=O)N,C13H16N4O,DB07232,D09692,veliparib,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
1,4645,1,3,4,uM,uM,2.612379,-2.542583,5.443890,2.058551,...,CC(C)(C1=NC(=CC=C1)N2C3=NC(=NC=C3C(=O)N2CC=C)N...,C27H32N8O2,DB11740,NaN,MK-1775,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
2,4649,1,4,4,uM,uM,3.866124,0.871119,6.244486,2.778483,...,CC(C)(C#N)C1=CC=C(C=C1)N2C3=C4C=C(C=CC4=NC=C3N...,C30H23N5O,DB11651,D10552,NVP-BEZ235,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
3,4653,1,5,4,uM,uM,0.589177,-16.482204,-4.214315,-4.724869,...,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,C19H25BN4O4,DB00188,D03150,bortezomib,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
4,4657,1,8,4,uM,uM,-0.756504,-5.749643,0.124410,-3.229846,...,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=NC(=NC(...,C22H26ClN7O2S,DB01254,D03658,dasatinib,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735


In [21]:
from collections import defaultdict

# Initialize dictionary with sets to avoid duplicates
drug_to_cell_lines = defaultdict(set)

# Iterate through each row
for _, row in summary_drugcomb.iterrows():
    drug1 = row['cmaps_name_drug1'] #row
    drug2 = row['cmaps_name_drug2'] #col
    cell = row['cell_line_name']
    
    drug_to_cell_lines[drug1].add(cell)
    drug_to_cell_lines[drug2].add(cell)

# Convert sets to lists
drug_to_cell_lines = {drug: list(cells) for drug, cells in drug_to_cell_lines.items()}


In [22]:
len(drug_to_cell_lines) ## there are 49 unique drugs the dictionary is okay 

49

Now let's iterate through the dictionary to:

* Iterate over each drug in your dictionary (drug_to_cell_lines).
* For each drug, check in sig_info how many signatures (sig_id) are associated with each of its cell lines.
* Save or print that count.

In [23]:
import pandas as pd

def count_and_list_signatures_per_drug_and_cellline(drug_to_cell_lines, sig_info):
    data = []

    for drug, cell_lines in drug_to_cell_lines.items():
        for cell in cell_lines:
            matching_sigs = sig_info[
                (sig_info["pert_iname"] == drug) &
                (sig_info["cell_id"] == cell)
            ]
            sig_ids = matching_sigs["sig_id"].unique().tolist()
            num_signatures = len(sig_ids)
            data.append({
                "drug": drug,
                "cell_line": cell,
                "num_signatures": num_signatures,
                "signature_ids": sig_ids
            })

    return pd.DataFrame(data)

In [24]:
#277x4
#126x4 porque ninguno vacio
drugs_cell_signature_names = count_and_list_signatures_per_drug_and_cellline(drug_to_cell_lines, sig_info)
drugs_cell_signature_names

,drug,cell_line,num_signatures,signature_ids
0,fluorouracil,MCF7,6,"[REP.A004_MCF7_24H:J19, REP.A004_MCF7_24H:J20,..."
1,fluorouracil,HT29,6,"[REP.A004_HT29_24H:J19, REP.A004_HT29_24H:J20,..."
2,fluorouracil,A375,6,"[REP.A004_A375_24H:J19, REP.A004_A375_24H:J20,..."
3,veliparib,HT29,18,"[LJP008_HT29_24H:A07, LJP008_HT29_24H:A08, LJP..."
4,veliparib,A375,18,"[LJP008_A375_24H:A07, LJP008_A375_24H:A08, LJP..."
...,...,...,...,...
121,vismodegib,HT29,6,"[REP.A017_HT29_24H:D01, REP.A017_HT29_24H:D02,..."
122,vismodegib,MCF7,6,"[REP.A017_MCF7_24H:D01, REP.A017_MCF7_24H:D02,..."
123,vandetanib,A549,6,"[LJP007_A549_24H:P13, LJP007_A549_24H:P14, LJP..."
124,vandetanib,HT29,12,"[LJP007_HT29_24H:P13, LJP007_HT29_24H:P14, LJP..."


there are some drugs-cell lines that have no enries in cmaps :(, we will have to take this down 

In [30]:
import pandas as pd
from cmapPy.pandasGEXpress import parse
from tqdm import tqdm

# Ruta al archivo GCTX
ruta_gctx = r"C:\Users\K6502\OneDrive\Desktop\TFM\datos\CLUE_data\CMAPS\\GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx"

In [31]:
def procesar_firma(row, ruta_gctx):
    from cmapPy.pandasGEXpress import parse
    drug = row["drug"]
    cell_line = row["cell_line"]
    sig_ids = row["signature_ids"]

    # Si no hay firmas asociadas, omitir este caso
    if len(sig_ids) == 0:
        return None, None 

    data_subset = parse.parse(ruta_gctx, cid=sig_ids).data_df

    mean_signature = data_subset.mean(axis=1)
    unique_col_name = f"{drug}_{cell_line}"
    mean_signature = mean_signature.rename(unique_col_name)
    metadata = {"drug": drug, "cell_line": cell_line, "sig_ids": sig_ids}

    return mean_signature, (unique_col_name, metadata)

In [32]:
# Diccionarios y listas para almacenar resultados
signature_dict = {}
metadata_dict = {}
mean_signatures = []

# Iterar sobre cada fila usando tqdm para barra de progreso
for _, row in tqdm(drugs_cell_signature_names.iterrows(), total=drugs_cell_signature_names.shape[0]):
    mean_sig, metadata_entry = procesar_firma(row, ruta_gctx)

    if mean_sig is not None:
        mean_signatures.append(mean_sig)
        col_name, metadata = metadata_entry
        metadata_dict[col_name] = metadata

# Concatenar todas las firmas promedio en un DataFrame final
final_signature_df = pd.concat(mean_signatures, axis=1)

# Convertir metadata_dict a DataFrame si lo quieres también
metadata_df = pd.DataFrame.from_dict(metadata_dict, orient='index')


The given path to the gctx file cannot be found. full_path: C:\Users\K6502\OneDrive\Desktop\TFM\datos\CLUE_data\CMAPS\\GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx
  0%|          | 0/126 [00:00<?, ?it/s]


Exception: The given path to the gctx file cannot be found. full_path: C:\Users\K6502\OneDrive\Desktop\TFM\datos\CLUE_data\CMAPS\\GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx

In [24]:
final_signature_df

,fluorouracil_A375,fluorouracil_HT29,fluorouracil_MCF7,veliparib_A375,veliparib_HT29,MK-1775_A375,MK-1775_HT29,NVP-BEZ235_A375,NVP-BEZ235_HT29,bortezomib_A375,...,ruxolitinib_HT29,ruxolitinib_MCF7,crizotinib_A549,crizotinib_HT29,crizotinib_MCF7,vismodegib_HT29,vismodegib_MCF7,vandetanib_A549,vandetanib_HT29,vandetanib_MCF7
rid,,,,,,,,,,,,,,,,,,,,,
780,-0.454034,0.556375,-0.241931,0.232655,0.132375,0.933442,-0.286527,0.571513,2.081833,4.519783,...,-0.189953,0.142635,0.132573,0.378890,0.788946,0.121903,-0.387006,0.389896,0.065462,1.088273
7849,-0.061263,-0.303017,-0.070943,-0.063150,0.073779,1.349271,-0.168490,1.029521,1.175859,4.240863,...,-0.476028,-0.356934,-0.133523,0.445199,0.191256,-0.795835,-0.416783,0.167681,0.123265,0.168304
2978,0.163443,-0.177575,-0.469085,-0.291728,-0.200203,0.148263,-0.440052,0.663628,0.232873,3.734846,...,-0.018869,-0.244178,0.431875,0.368290,-0.139743,0.427223,0.264359,-0.225446,-0.251624,0.193593
2049,0.149307,-0.432942,0.218524,-0.215559,0.185203,0.125775,-0.709749,0.426363,0.794531,1.540876,...,-0.133998,0.151536,0.015063,0.425805,0.030804,0.001623,-0.120179,0.104658,0.269996,0.241960
2101,0.312194,0.065542,-0.502032,-0.035518,0.273005,1.115665,0.874413,0.757554,0.614271,0.313735,...,0.122460,0.471076,-0.161662,-0.022422,-0.229070,0.172411,-0.111053,-0.251501,0.294118,0.071981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4034,0.546573,0.257167,0.403539,-0.161075,0.310619,-0.341197,0.436750,-0.063948,-0.163597,0.337468,...,-0.222218,-0.041184,-0.149687,0.138460,-0.356140,-0.317525,0.085895,-0.088846,-0.074151,-0.530467
399664,-0.121906,0.036458,-0.018679,0.023309,-0.003192,-0.900052,0.388490,-0.698817,-0.282646,0.906080,...,0.336216,0.165212,-0.527911,0.257806,0.135142,-0.219650,0.012699,-0.160071,0.404592,-0.176600
54869,-0.661758,-0.662400,0.044976,-0.002781,-0.081142,0.534846,0.188565,0.202625,0.675649,4.216597,...,0.207338,-0.224512,0.122875,0.358292,0.260596,-0.139046,-0.125041,0.045230,-0.191759,0.245778


In [25]:
metadata_df 

,drug,cell_line,sig_ids
fluorouracil_A375,fluorouracil,A375,"[REP.A004_A375_24H:J19, REP.A004_A375_24H:J20,..."
fluorouracil_HT29,fluorouracil,HT29,"[REP.A004_HT29_24H:J19, REP.A004_HT29_24H:J20,..."
fluorouracil_MCF7,fluorouracil,MCF7,"[REP.A004_MCF7_24H:J19, REP.A004_MCF7_24H:J20,..."
veliparib_A375,veliparib,A375,"[LJP008_A375_24H:A07, LJP008_A375_24H:A08, LJP..."
veliparib_HT29,veliparib,HT29,"[LJP008_HT29_24H:A07, LJP008_HT29_24H:A08, LJP..."
...,...,...,...
vismodegib_HT29,vismodegib,HT29,"[REP.A017_HT29_24H:D01, REP.A017_HT29_24H:D02,..."
vismodegib_MCF7,vismodegib,MCF7,"[REP.A017_MCF7_24H:D01, REP.A017_MCF7_24H:D02,..."
vandetanib_A549,vandetanib,A549,"[LJP007_A549_24H:P13, LJP007_A549_24H:P14, LJP..."
vandetanib_HT29,vandetanib,HT29,"[LJP007_HT29_24H:P13, LJP007_HT29_24H:P14, LJP..."


#### Ahora filtramos por los landmark genes

In [27]:
# Asegurate de que geneInfoSM esté bien definido
geneInfoSM = gene_info

# Extraer los IDs de landmark genes como string (para que coincidan con el índice del DF)
lm_gene_ids = geneInfoSM[geneInfoSM["pr_is_lm"] == 1]["pr_gene_id"].astype(str)

# Filtrar el DataFrame final_signature_df para quedarte solo con las filas de landmark genes
final_signature_df_lm = final_signature_df.loc[final_signature_df.index.isin(lm_gene_ids)]

In [28]:
# Revisar el shape
final_signature_df_lm.shape


(978, 126)

In [29]:
final_signature_df_lm

,fluorouracil_A375,fluorouracil_HT29,fluorouracil_MCF7,veliparib_A375,veliparib_HT29,MK-1775_A375,MK-1775_HT29,NVP-BEZ235_A375,NVP-BEZ235_HT29,bortezomib_A375,...,ruxolitinib_HT29,ruxolitinib_MCF7,crizotinib_A549,crizotinib_HT29,crizotinib_MCF7,vismodegib_HT29,vismodegib_MCF7,vandetanib_A549,vandetanib_HT29,vandetanib_MCF7
rid,,,,,,,,,,,,,,,,,,,,,
780,-0.454034,0.556375,-0.241931,0.232655,0.132375,0.933442,-0.286527,0.571513,2.081833,4.519783,...,-0.189953,0.142635,0.132573,0.378890,0.788946,0.121903,-0.387006,0.389896,0.065462,1.088273
7849,-0.061263,-0.303017,-0.070943,-0.063150,0.073779,1.349271,-0.168490,1.029521,1.175859,4.240863,...,-0.476028,-0.356934,-0.133523,0.445199,0.191256,-0.795835,-0.416783,0.167681,0.123265,0.168304
6193,0.458396,-0.112558,-0.521271,-0.130233,0.240076,-0.559515,-0.691441,0.927662,-1.006166,-2.338197,...,0.106898,0.412513,-0.578974,-0.589351,-0.290396,-0.479794,-0.652074,-0.698794,-0.057060,-0.178580
23,-0.013901,-0.116283,-0.102935,0.343411,0.004642,1.558415,0.744736,-0.759341,-0.152758,5.891055,...,0.699038,0.009236,0.040954,-0.033205,0.439969,0.439828,0.166281,0.440294,0.048169,0.079787
9552,0.224778,-0.288575,-0.312377,-0.168420,-0.464064,-0.131803,0.339135,1.264653,1.243738,4.625013,...,-0.184445,-0.047064,-0.572966,0.740158,0.644811,-0.560352,0.042487,0.323581,-0.176176,0.396500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5467,-0.046469,0.228017,0.113124,0.133784,0.095374,0.062771,-0.317576,-0.048354,0.401185,4.034200,...,-0.396192,-0.097672,-0.094230,-0.032198,-0.083014,0.073672,-0.075906,0.203334,0.228294,0.555357
2767,0.196210,0.625300,-0.216124,0.305656,0.241276,-0.153740,0.368022,-0.786243,-0.051111,-0.138223,...,0.289809,0.601744,-0.075359,-0.127368,0.217210,0.493079,0.677853,0.333448,0.201346,-0.292288
23038,0.731427,0.379617,0.383257,0.105282,-0.039592,0.137195,-0.697365,0.905309,1.814131,-4.738132,...,-0.343929,0.556174,-0.219226,1.374665,0.621156,-0.426728,0.118265,0.778324,0.784130,0.217076


In [30]:
final_signature_df_lm.to_csv(r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\drug_signatures_drugcomb_cmaps_combination.csv")